In [11]:
%matplotlib inline

from matplotlib import pyplot as plt
from matplotlib import patches as mpatches
import numpy as np
import pandas as pd
import math
import sys

In [12]:
data = pd.read_csv("data.csv", index_col="Ind.", sep=",")

In [13]:
P_A = 1e5 # atmosphere pressure WARNING (can be not 1e5)
R = 8.314

def init():
    global P_A
    sys.stderr.write("P_a = %f" % (P_A))
    
    x = lambda T: T / 1e4
    
    data["sigmaN2"] = (data["sigma"] + data["sigma"]["N2"]) / 2.0
    data["epsilonN2"] = np.sqrt(data["epsilon"] * data["epsilon"]["N2"])
    data["muN2"] =  2 * data["mu"]["N2"] * data["mu"] / (data["mu"]["N2"] + data["mu"])
    
    def generate(subst):
        data.loc[subst, "omega"] = lambda T: 1.074 * np.power(T / data["epsilonN2"][subst], -0.1604)        
        data.loc[subst, "Phi"] = lambda T: \
                      data["f1"][subst] \
                    + data["f2"][subst] * np.log(x(T)) \
                    + data["f3"][subst] * np.power(x(T), -2.0) \
                    + data["f4"][subst] * np.power(x(T), -1.0) \
                    + data["f5"][subst] * x(T) \
                    + data["f6"][subst] * np.power(x(T), 2.0) \
                    + data["f7"][subst] * np.power(x(T), 3.0)
                    
        data.loc[subst, "G"] = lambda T: data["H(298)"][subst] - data["Phi"][subst](T) * T
        
        data.loc[subst, "D"] = lambda T: 2.628e-2 * np.power(T, 3.0 / 2.0) / \
                        (P_A * data["sigmaN2"][subst] * data["omega"][subst](T) * np.sqrt(data["muN2"][subst]))
            
    closure = [generate(subst) for subst in data.index]
    
init()

P_a = 100000.000000

In [22]:
#             case 1: dG = 2 * AL.G(T) + 2 * H_CL.G(T) - 2 * AL_CL.G(T) - H2.G(T); mult = 1 / P_A; break;
#             case 2: dG = AL.G(T) + 2 * H_CL.G(T) - AL_CL2.G(T) - H2.G(T); mult = 1; break;
#             case 3: dG = 2 * AL.G(T) + 6 * H_CL.G(T) - 2 * AL_CL3.G(T) - 3 * H2.G(T); mult = P_A; break;
#             case 4: dG = 2 * GA.G(T) + 2 * H_CL.G(T) - 2 * GA_CL.G(T) - H2.G(T); mult = 1 / P_A; break;
#             case 5: dG = GA.G(T) + 2 * H_CL.G(T) - GA_CL2.G(T) - H2.G(T); mult = 1; break;
#             case 6: dG = 2 * GA.G(T) + 6 * H_CL.G(T) - 2 * GA_CL3.G(T) - 3 * H2.G(T); mult = P_A; break;
#             //case 7: dG = AL_CL.G(T) + N_H3.G(T) - AL_N.G(T) - H_CL.G(T) - H2.G(T); break;
#             //case 8: dG = 2 * AL_CL2.G(T) + 2 * N_H3.G(T) - 2 * AL_N.G(T) - 4 * H_CL.G(T) - H2.G(T); break;
#             case 9: dG = AL_CL3.G(T) + N_H3.G(T) - AL_N.G(T) - 3 * H_CL.G(T); mult = 1 / P_A; break;
#             case 10: dG = GA_CL.G(T) + N_H3.G(T) - GA_N.G(T) - H_CL.G(T) - H2.G(T); mult = 1; break;
#             //case 11: dG = 2 * GA_CL2.G(T) + 2 * N_H3.G(T) - 2 * GA_N.G(T) - 4 * H_CL.G(T) - H2.G(T); break;
#             //case 12: dG = GA_CL3.G(T) + N_H3.G(T) - GA_N.G(T) - 3 * H_CL.G(T); break;
#             default: return Double.NaN; //throw new IllegalArgumentException("Invalid equation number: " + number);

K = [lambda T: np.exp(-(2 * data["G"]["Al"](T) 
                        + 2 * data["G"]["HCl"](T) 
                        - 2 * data["G"]["AlCl"](T) 
                        - data["G"]["H2"](T)) / (R * T)) / P_A,
     lambda T: np.exp(-(data["G"]["Al"](T) 
                        + 2 * data["G"]["HCl"](T) 
                        - data["G"]["AlCl2"](T) 
                        - data["G"]["H2"](T)) / (R * T)),
     lambda T: np.exp(-(2 * data["G"]["Al"](T) 
                        + 6 * data["G"]["HCl"](T) 
                        - 2 * data["G"]["AlCl3"](T) 
                        - 3 * data["G"]["H2"](T)) / (R * T)) * P_A,
     lambda T: np.exp(-(2 * data["G"]["Ga"](T) 
                        + 2 * data["G"]["HCl"](T) 
                        - 2 * data["G"]["GaCl"](T) 
                        - data["G"]["H2"](T)) / (R * T)) / P_A,
     lambda T: np.exp(-(data["G"]["Ga"](T) 
                        + 2 * data["G"]["HCl"](T) 
                        - data["G"]["GaCl2"](T) 
                        - data["G"]["H2"](T)) / (R * T)),
     lambda T: np.exp(-(2 * data["G"]["Ga"](T) 
                        + 6 * data["G"]["HCl"](T) 
                        - 2 * data["G"]["GaCl3"](T) 
                        - 3 * data["G"]["H2"](T)) / (R * T)) * P_A,
     lambda T: 0,
     lambda T: 0,
     lambda T: np.exp(-(data["G"]["AlCl3"](T) 
                        + data["G"]["NH3"](T) 
                        - data["G"]["AlN"](T) 
                        - 3 * data["G"]["HCl"](T)) / (R * T)) / P_A,
     lambda T: np.exp(-(data["G"]["GaCl"](T)  
                        + data["G"]["NH3"](T) 
                        - data["G"]["GaN"](T) 
                        - data["G"]["HCl"](T) 
                        - data["G"]["H2"](T)) / (R * T)),
     lambda T : 0,
     lambda T : 0
    ]


In [31]:
# Task1
# 0 -- AlCl
# 1 -- AlCl2
# 2 -- AlCl3
# 3 -- H2
# 4 -- HCl

def solve(T):
    # Jacobian
    J = \
    [ 
        [ # 0 :::  (PHCl ** 2) - K0 * (PAlCl ** 2) * PH2 = 0
            lambda x: 0 - 2 * K[0](T) * x[0] * x[3], 
            lambda x: 0, 
            lambda x: 0, 
            lambda x: 0 - K[0](T) * x[0] * x[0], 
            lambda x: 2 * x[4]
        ],
    
        [ # 1 ::: (PHCl ** 2) - K1 * PAlCl2 * PH2 = 0
            lambda x: 0, 
            lambda x: 0 - K[1](T) * x[3], 
            lambda x: 0, 
            lambda x: 0 - K[1](T) * x[1], 
            lambda x: 2 * x[4] 
        ],
        
        [ # 2 ::: (PHCl ** 6) - K2 * (PAlCl3 ** 2) * (PH2 ** 3) = 0
            lambda x: 0,
            lambda x: 0,
            lambda x: 0 - 2 * K[2](T) * x[2] * (x[3] ** 3),
            lambda x: 0 - 3 * K[2](T) * (x[2] ** 2) * (x[3] ** 2),
            lambda x: 6 * (x[4] ** 5)
        ],
    
        [ # 3 ::: DHCl(PgHCl - PeHCl) + 2 * DH2 * (PgH2 - PeH2) = 0
            lambda x: 0,
            lambda x: 0,
            lambda x: 0,
            lambda x: -2 * data["D"]["H2"](T),
            lambda x: -1 * data["D"]["HCl"](T),        
        ],
        
        [    # 4 ::: DAlCl(Pg(AlCl) - Pe(AlCl)) 
            #   + 2 * D(AlCl2)(Pg(AlCl2) - Pe(AlCl2)) 
            #   + 3 * D(AlCl3) * (Pg(AlCl3) - Pe(AlCl3)) 
            #   + D(HCl) * (Pg(HCl) - Pe(HCl)) = 0

            lambda x: -data["D"]["AlCl"](T),
            lambda x: -2 * data["D"]["AlCl2"](T),
            lambda x: -3 * data["D"]["AlCl3"](T),
            lambda x: 0,
            lambda x: -data["D"]["HCl"](T)
        ]
    ]

solve(365)